# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('/Users/miguelmoreno/Desktop/IRONHACK/week_3_day_1/lab-pandas-calculations/your-code/Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [22]:
# your code here
#SUB PROBLEM 1: HOW TO AGGREGATE THE AMOUNT_SPENT FOR UNIQUE CUSTOMERS?
agg_amount_by_customer = orders.groupby("CustomerID").agg({
    "amount_spent": "sum",   # aggregate the amount_spent per customer_id
    "Country": "first"
}).reset_index()

#MERGED BACK TO THE ORIGINAL DATASET TO HAVE ALL THE VALUE STILL AVAILABLE FOR FUTURE CALCULATIONS.
agg_amount_by_customer

,CustomerID,amount_spent,Country
0,12346,77183.60,United Kingdom
1,12347,4310.00,Iceland
2,12348,1797.24,Finland
3,12349,1757.55,Italy
4,12350,334.40,Norway
...,...,...,...
4334,18280,180.60,United Kingdom
4335,18281,80.82,United Kingdom
4336,18282,178.05,United Kingdom
4337,18283,2094.88,United Kingdom


In [27]:
#SUB PROBLEM 2: HOW TO SELECT CUSTOMERS WHOSE AGGREGATED amount_spent IS IN A GIVEN QUANTILE RANGE?
#VIP CUSTOMERS ABOVE THE 95% 
#PREFERRED CUSTOMERS BETWEEN 75% AND 95%

#SORT TABLE BY ASCENDING (ascending=True)
sorted_amount_asc = agg_amount_by_customer.sort_values(by='amount_spent',ascending=True) #4339 ROWS, $8,911,407.904 total_amount
#CREATE A NEW COLUMN WITH TOTAL REVENUE 
sorted_amount_asc['total_amount'] = sorted_amount_asc['amount_spent'].sum()
#CREATE A NEW COLUMN TO GET THE PERCENTAGE OF THE TOTAL_REVENUE
sorted_amount_asc['quartile'] = (sorted_amount_asc['amount_spent'] / sorted_amount_asc['total_amount'])*100
#APPLIED cumsum() FUNCTION TO GET THE SUM OF THE QUARTILE
sorted_amount_asc['sumquartile'] = sorted_amount_asc['quartile'].cumsum()
#MOSTRAR LA TABLA
sorted_amount_asc


,CustomerID,amount_spent,Country,total_amount,quartile,sumquartile
685,13256,0.00,United Kingdom,8911407.904,0.000000,0.000000
3218,16738,3.75,United Kingdom,8911407.904,0.000042,0.000042
1794,14792,6.20,United Kingdom,8911407.904,0.000070,0.000112
3015,16454,6.90,United Kingdom,8911407.904,0.000077,0.000189
4099,17956,12.75,United Kingdom,8911407.904,0.000143,0.000332
...,...,...,...,...,...,...
1880,14911,143825.06,EIRE,8911407.904,1.613943,89.868193
3009,16446,168472.50,United Kingdom,8911407.904,1.890526,91.758720
3729,17450,194550.79,United Kingdom,8911407.904,2.183166,93.941885
4202,18102,259657.30,United Kingdom,8911407.904,2.913763,96.855648


In [ ]:
#HACER LA CLASIFICACION DE CLIENTES EN BASE A sumquartile

def categorize(row):
    if row['sumquartile'] >= 95:
        return 'VIP'
    elif 75 <= row['sumquartile'] < 95:
        return 'Preferred Customers'
    else:
        return 'Regular Customers'
    
#apply() PARA APLICAR A TODAS LAS FILAS O COLUMNAS, axis=1 PARA INDICAR A PANDAS QUE APLIQUE LA FUNCION FILA POR FILA(NO COLUMNA)
sorted_amount_asc['category'] = sorted_amount_asc.apply(categorize, axis=1) 

sorted_amount_asc

,CustomerID,amount_spent,Country,total_amount,quartile,sumquartile,category
685,13256,0.00,United Kingdom,8911407.904,0.000000,0.000000,Regular Customers
3218,16738,3.75,United Kingdom,8911407.904,0.000042,0.000042,Regular Customers
1794,14792,6.20,United Kingdom,8911407.904,0.000070,0.000112,Regular Customers
3015,16454,6.90,United Kingdom,8911407.904,0.000077,0.000189,Regular Customers
4099,17956,12.75,United Kingdom,8911407.904,0.000143,0.000332,Regular Customers
...,...,...,...,...,...,...,...
1880,14911,143825.06,EIRE,8911407.904,1.613943,89.868193,Preferred Customers
3009,16446,168472.50,United Kingdom,8911407.904,1.890526,91.758720,Preferred Customers
3729,17450,194550.79,United Kingdom,8911407.904,2.183166,93.941885,Preferred Customers
4202,18102,259657.30,United Kingdom,8911407.904,2.913763,96.855648,VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [55]:
# your code here
#assing() SE USA PARA CREAR UNA NUEVA COLUMNA O MODIFICAR UNA EXISTENTE
#query() SE USA PARA HACER FILTROS

top_vip_customer = (
    sorted_amount_asc.assign(CustomerID=sorted_amount_asc['CustomerID'].astype(str))
    .query("category == 'VIP'")
    .sort_values(by='amount_spent', ascending=False)
    .query("amount_spent == amount_spent.max()")
)

top_vip_customer

,CustomerID,amount_spent,Country,total_amount,quartile,sumquartile,category
1690,14646,280206.02,Netherlands,8911407.904,3.144352,100.0,VIP


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [58]:

top_vip_cust_VIP_Preferred = (
    sorted_amount_asc.query("category in ['VIP', 'Preferred Customers']")
    .sort_values(by='amount_spent', ascending=False)
    .groupby('category')
    .head(1)
)

top_vip_cust_VIP_Preferred

,CustomerID,amount_spent,Country,total_amount,quartile,sumquartile,category
1690,14646,280206.02,Netherlands,8911407.904,3.144352,100.000000,VIP
3729,17450,194550.79,United Kingdom,8911407.904,2.183166,93.941885,Preferred Customers
